# Entity Explorer - Account
 <details>
     <summary>&nbsp;<u>Details...</u></summary>

 **Notebook Version:** 1.0<br>
 **Python Version:** Python 3.6 (including Python 3.6 - AzureML)<br>
 **Required Packages**: kqlmagic, msticpy, pandas, numpy, matplotlib, networkx, ipywidgets, ipython, dnspython, ipwhois, folium, maxminddb_geolite2<br>
 **Platforms Supported**:
 - Azure Notebooks Free Compute
 - Azure Notebooks DSVM
 - OS Independent

 **Data Sources Required**:
 - Log Analytics - SecurityAlert, SecurityEvent, HuntingBookmark, Syslog, AAD SigninLogs, AzureActivity, OfficeActivity, ThreatIndicator
 - (Optional) - VirusTotal, AlienVault OTX, IBM XForce, Open Page Rank, (all require accounts and API keys)
 </details>

 Brings together a series of queries and visualizations to help you determine the security state of an Account. The account can be a Windows or Linux account or an Azure Active Directory/Office 365 account.

The Notebook contains sections for reviewing activity for Host accounts (Linux and Windows) and for Azure Active Directory accounts. It also has a general section that looks for related items independent of the account type.

<!DOCTYPE html>
<html>
  <head>
  </head>
  <body>
    <h1>Contents<span class="tocSkip"></span></h1>
    <div class="toc">
      <ul class="toc-item">
        <li><span><a href="#Hunting-Hypothesis">Hunting Hypothesis</a></span></li>
        <li><span><a href="#Notebook-initialization">Notebook initialization</a></span></li>
        <li><span><a href="#Enter-account-name-and-query-time">Enter account
              name and query time</a></span></li>
        <li><span><a href="#Run-the-main-account_summary-function">Run the main
              account_summary function</a></span></li>
        <li><span><a href="#Retrieve-additional-data-for-the-selected-account">Retrieve
              additional data for the selected account</a></span></li>
        <li><span><a href="#Browse-alerts">Browse alerts</a></span></li>
        <li><span><a href="#Browse-other-event-data">Browse other event data</a></span></li>
        <li><span><a href="#Additional-properties-and-methods-of-the-result-object">
              Additional properties and methods of the result object<br>
            </a></span></li>
        <li><span><a href="#Viewing-the-Result-class">Viewing the Result class</a></span></li>
        <li><span><a href="#Use-other-notebooklets-and-pivots-functions-to-drill-down-on-other-entities">
              Use other notebooklets and pivots functions to drill down on other
              entities</a></span></li>
        <li><span><a href="#More-information">More information</a></span></li>
      </ul>
    </div>
    
  </body>
</html>


## Hunting Hypothesis
Our broad initial hunting hypothesis is that a we have received account name entity which is suspected to be compromised and is being used malicious manner in internal networks, we will need to hunt from a range of different positions to validate or disprove this hypothesis.

---
# Notebook initialization
This should complete without errors. If you encounter errors or warnings look at the following notebooks:

- <a href="https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb">Getting Started Notebook</a>
- [TroubleShootingNotebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/TroubleShootingNotebooks.ipynb)
- [ConfiguringNotebookEnvironment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)

<details>
    <summary>&nbsp;<u>Details...</u></summary>
The next cell:
- Checks for the correct Python version
- Checks versions and optionally installs required packages
- Imports the required packages into the notebook
- Sets a number of configuration options.

If you are running in the Azure Sentinel Notebooks environment (Azure Notebooks or Azure ML) you can run live versions of these notebooks:
- [Getting Started](./A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb)
- [Run TroubleShootingNotebooks](./TroubleShootingNotebooks.ipynb)
- [Run ConfiguringNotebookEnvironment](./ConfiguringNotebookEnvironment.ipynb)

You may also need to do some additional configuration to successfully use functions such as Threat Intelligence service lookup and Geo IP lookup. 
There are more details about this in the `ConfiguringNotebookEnvironment` notebook and in these documents:
- [msticpy configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [Threat intelligence provider configuration](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html#configuration-file)
</details>

In [1]:
from pathlib import Path
from IPython.display import display, HTML

REQ_PYTHON_VER=(3, 6)
REQ_MSTICPY_VER=(1, 0, 0)

display(HTML("<h3>Starting Notebook setup...</h3>"))
if Path("./utils/nb_check.py").is_file():
    from utils.nb_check import check_versions
    check_versions(REQ_PYTHON_VER, REQ_MSTICPY_VER, extras=["azsentinel"])

# You may need to manually install msticpy with
# !pip install msticpy[azsentinel]

from msticpy.nbtools import nbinit
nbinit.init_notebook(
    namespace=globals(),
    additional_packages=["msticnb>=0.2"],
);


msticpy version installed: 1.0.0rc4 latest published: 0.9.0
Latest version is installed.
Processing imports....
All packages are already installed
Checking configuration....
No errors found.
No warnings found.
Exception while checking configuration:
cannot unpack non-iterable NoneType object
Setting notebook options....


### Get Workspace and Authenticate
<details>
    <summary>&nbsp;<u>Details...</u></summary>
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.<br>
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. <br>
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. <br>
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.<br>

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

Note: you may occasionally see a JavaScript warning displayed at the end of the authentication - you can safely ignore this.<br>
On successful authentication you should see a ```popup schema``` button.
To find your Workspace Id go to [Log Analytics](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.
</details>

In [2]:
# Authentication
# ws_config = WorkspaceConfig()
qry_prov = QueryProvider(data_environment="LogAnalytics")
qry_prov.connect(WorkspaceConfig())

Please wait. Loading Kqlmagic extension...


#### Authentication and Configuration problems?

If you are having problems, expand the details section below
<br>
<details>
    <summary>Click for details about configuring your authentication parameters</summary>
    
    
The notebook is expecting your Azure Sentinel Tenant ID and Workspace ID to be configured in one of the following places:
- `config.json` in the current folder
- `msticpyconfig.yaml` in the current folder or location specified by `MSTICPYCONFIG` environment variable.
    
For help with setting up your `config.json` file (if this hasn't been done automatically) see the [`ConfiguringNotebookEnvironment`](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb) notebook in the root folder of your Azure-Sentinel-Notebooks project. This shows you how to obtain your Workspace and Subscription IDs from the Azure Sentinel Portal. You can use the SubscriptionID to find your Tenant ID). To view the current `config.json` run the following in a code cell.

```%pfile config.json```

For help with setting up your `msticpyconfig.yaml` see the [Setup](#Setup) section at the end of this notebook and the [ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)
</details>

## Import notebooklets and initialize pivot functions

These are needed for many of the notebook operations

In [3]:
import msticnb as nb

pivot = Pivot(globals())
nb.init(query_provider=qry_prov)

Notebooklets: 8 notebooklets loaded.
Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank
Notebooklets: Loaded providers: AzureSentinel, geolitelookup, tilookup


# Enter account name and query time window
Type the account name that you want to search for and the time bounds over which you want to search. 

You can specify the account as:

- a simple user name (e.g. `alice`)
- a user principal name (`alice@contoso.com`)
- a qualified windows user name `mydomain\alice`

In the second two cases the domain qualifier will be stripped off before the search. The search is not case sensitive and will match full substrings. E.g. `bob` will match `domain\bob` and `bob@contoso.com` but not `bobg` or `bo`.

In [4]:
account_txt = nbwidgets.GetText(prompt='Enter the Account name to search for:')
md("<hr>")
display(account_txt)
md("<hr>")
pivot.edit_query_time()

Text(value='', description='Enter the Account name to search for:', layout=Layout(width='50%'), style=Descript…

# Run the main *account_summary* function

If more than one matching account name is found, all matches will be shown. 

You can select each one to view more details about the account.

> Note: this is run as a pivot function from the Account entity.
> You can also run it directly as a Notebooklet function by 
> creating and instance of `nb.nblt.azsent.account.AccountSummary()`
> and executing the `run()` method

```python
>>> acct_nb = nb.nblt.azsent.account.AccountSummary()
>>> acct_result = acct_nb.run(pivot.timespan, "account_name")  # subtitute the "account_name" with the name to search
```

In [5]:
Account = entities.Account
acc_result = Account.nblt.account_summary(value=account_txt.value)

Loading BokehJS ...

Account(Name=seb@seccxp.ninja, UPNSuffix=seccxp.ninja, AadTenantId=4b2462a4-bbee-495a-a0...)

,TimeGenerated,TenantId,SourceSystem,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationDetails,AuthenticationMethodsUsed,AuthenticationProcessingDetails,AuthenticationRequirement,...,RiskLevelAggregated,RiskLevelDuringSignIn,RiskState,ResourceDisplayName,ResourceIdentity,ServicePrincipalId,ServicePrincipalName,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,UserType,FlaggedForReview,IPAddressFromResourceProvider,SignInIdentifier,SignInIdentifierType,ResourceTenantId,HomeTenantId,Type,AccountName,Source
0,2021-03-26 17:29:10.936000+00:00,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,60ea5f31-8e4e-454f-b2e5-da79b4477e24,Microsoft.aadiam,Microsoft.aadiam,,Admin,4,BE,,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,"[\r\n {\r\n ""authenticationStepDateTime"": ""2021-03-26T17:29:10.9363339+00:00"",\r\n ""authe...",,"[\r\n {\r\n ""key"": ""Azure AD App Authentication Library"",\r\n ""value"": ""Family: ASP.NET L...",singleFactorAuthentication,...,none,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,,,{'errorCode': 0},,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.438...",Admin,bdb72692-2275-412b-8af0-ea21c2ebd174,seb@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,seb@seccxp.ninja,AzureActiveDirectory


# Retrieve additional data for the selected account

The result returned from the last cell has a number of properties and methods that you can use
to retrieve and view further information.

The main one for this notebooklet is `get_additional_data`.

In [6]:
acc_result.get_additional_data()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

OperationCount  \
UserPrincipalName Type           IPAddress                AppResourceProvider                          UserType                   
seb@seccxp.ninja  OfficeActivity 104.209.247.23           Exchange                                     Regular                4   
                                 109.88.218.99            Exchange                                     Regular                2   
                                 157.56.162.53            SharePoint                                   Regular               75   
                                 2603:10b6:4:2:cafe::e8   Exchange                                     Regular                1   
                                 2603:10b6:4:4a:cafe::70  Exchange                                     Regular                1   
                                 2603:10b6:4:ae:cafe::d7  Exchange                                     Regular                1   
                                 2603:10b6:5:177:cafe::9  Exchange                                     Regular                1   
                                 2603:10b6:5:18f:cafe::54 Exchange                                     Regular                1   
                                 2603:10b6:5:1f4:cafe::29 Exchange                                     Regular                1   
                                 2603:10b6:5:1f4:cafe::72 Exchange                                     Regular                1   
                                 2603:10b6:5:251::12      Exchange                                     Regular                1   
                                 2603:10b6:5:25e::9       Exchange                                     Regular                1   
                                 2603:10b6:5:296:cafe::6d Exchange                                     Regular                1   
                                 2603:10b6:5:336:cafe::6e Exchange                                     Regular                1   
                                 2603:10b6:5:3b9:cafe::47 Exchange                                     Regular                1   
                                 40.83.137.154            OneDrive                                     Regular              120   
                                 40.91.208.65             Exchange                                     Regular               72   
                                 52.154.174.165           Exchange                                     Regular               19   
                                 52.154.174.167           Exchange                                     Regular                6   
                                 52.225.101.225           Exchange                                     Regular               24   
                  SigninLogs     109.88.218.99            Azure Advanced Threat Protection             Member                 2   
                                                          Azure Portal                                 Member                 2   
                                                          Microsoft 365 Security and Compliance Center Member                 2   
                                                          Microsoft Cloud App Security                 Member                 5   
                                                          Office 365 Exchange Online                   Member                 1   
                                                          Office365 Shell WCSS-Client                  Member                 6   

                                                                                                                               OperationTypes  \
UserPrincipalName Type           IPAddress                AppResourceProvider                          UserType                                 
seb@seccxp.ninja  OfficeActivity 104.209.247.23           Exchange                                     Regular                         [Send]   
         

# Browse alerts
If there are any alerts referencing this account name they can be viewed
by calling the `acc_result.browse_alerts()` function.

In [7]:
acc_result.notebooklet.browse_alerts()

,0
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
TimeGenerated,2021-03-24 09:15:20.433000+00:00
AlertDisplayName,Azure Security Center unusual storage access
AlertName,Azure Security Center unusual storage access
Severity,Medium
Description,Azure Security Center unusual storage access
ProviderName,ASI Scheduled Alerts
VendorName,Microsoft
VendorOriginalId,55fd2263-0641-43c6-b2a2-09e4f1b12fd0
SystemAlertId,9d022478-1ab0-6619-4481-6fed372878d9


# Browse other event data

You can use a simple view to make it easier to see the details of individual events
by calling the "view_events" method.

You need to supply the name of the result attribute that you want to view plus
one or more summary columns (as a list of strings).

In [8]:
acc_result.view_events(attrib="azure_activity", summary_cols=["SourceSystem", "Operation", "IPAddress"])

,239
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
SourceSystem,OfficeActivityManager
TimeGenerated,2021-03-24 15:44:35+00:00
ResourceId,https://seccxpninja-my.sharepoint.com/personal/seb_seccxp_ninja/Documents/Private/SSN demo file.txt
Operation,FileModified
AppResourceProvider,OneDrive
IPAddress,40.83.137.154
UserId,seb@seccxp.ninja
UserPrincipalName,seb@seccxp.ninja
UserType,Regular


## Browse events with alternative sorting
You can pass a DatFrame to view events instead of an attribute name.

This means that you can apply sorting or filtering of the data before viewing it.
Here is an example sorting by IPAddress.

In [9]:
acc_result.view_events(
    data=acc_result.azure_activity.sort_values("IPAddress"),
    summary_cols=["SourceSystem", "Operation", "IPAddress"]
)

,350
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
SourceSystem,OfficeActivityManager
TimeGenerated,2021-03-24 08:07:09+00:00
Operation,Send
AppResourceProvider,Exchange
AppId,7ab7862c-4c57-491e-8a45-d52a7e023983
IPAddress,104.209.247.23
UserId,seb@seccxp.ninja
UserPrincipalName,seb@seccxp.ninja
UserType,Regular


# Additional properties and methods of the result object

These are static properties - usually DataFrames or visualizations. 
You can access each of these to see or manipulate the retrieved data.

To see help on the available attributes type:
```python
>>> help(acc_result)
```
To see the available methods type:
```python
>>> acc_result.list_methods()
```
> Note, for the AccountSummary notebooklet, the two main data retrieval methods are:<br>
> - run<br>
> - get_additional_data<br>
> There are several other methods that allow you to view individual plots
> or subsets of the data (such as alerts).

To view help on a specific method type:
```python
>>> help(acc_result.method_name())
```

In [10]:
acc_result.display_alert_timeline()

Loading BokehJS ...

Column(id='2722', ...)

# Viewing the Result class
You can view all of the data in the results class by "running" it in a cell

> Note: due to the way Jupyter display Javascript objects the plots may
> appear out of order.

In [11]:
acc_result

# Use other notebooklets and pivots functions to drill down on other entities

You may want to drill down on other entities in the Account data.
You can use methods of the IpAddress or Host entities, for example,
to look at these in more detail.

In [12]:
IpAddress = entities.IpAddress
ip_result = IpAddress.nblt.ip_address_summary("157.56.162.53")

,nir,asn_registry,asn,asn_cidr,asn_country_code,asn_date,asn_description,query,nets,raw,referral,raw_referral
0,None,arin,8075,157.56.0.0/16,US,1994-04-28,"MICROSOFT-CORP-MSN-AS-BLOCK, US",157.56.162.53,"{'cidr': '157.60.0.0/16, 157.54.0.0/15, 157.56.0.0/14', 'name': 'MSFT-GFS', 'handle': 'NET-157-5...",None,None,None


GeoLocation(CountryCode=US, CountryName=United States, State=California, City=San Jose, Long...)

,Ioc,IocType,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
OTX,157.56.162.53,ipv4,None,OTX,True,information,"{'pulse_count': 0, 'sections_available': ['general', 'geo', 'reputation', 'url_list', 'passive_d...","{'whois': 'http://whois.domaintools.com/157.56.162.53', 'reputation': 0, 'indicator': '157.56.16...",https://otx.alienvault.com/api/v1/indicators/IPv4/157.56.162.53/general,0
OPR,157.56.162.53,ipv4,None,OPR,False,information,IoC type ipv4 not supported.,None,None,1
Tor,157.56.162.53,ipv4,None,Tor,True,information,Not found.,None,https://check.torproject.org/exit-addresses,0
VirusTotal,157.56.162.53,ipv4,None,VirusTotal,True,information,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'detected_urls': [], 'positives': 0}","{'detected_urls': [], 'asn': 8075, 'country': 'US', 'response_code': 1, 'as_owner': 'Microsoft C...",https://www.virustotal.com/vtapi/v2/ip-address/report,0
XForce,157.56.162.53,ipv4,None,XForce,True,warning,"{'score': 1, 'cats': {}, 'categoryDescriptions': {}, 'reason': 'Regional Internet Registry', 're...","{'ip': '157.56.162.53', 'history': [{'created': '2012-03-22T07:26:00.000Z', 'reason': 'Regional ...",https://api.xforce.ibmcloud.com/ipr/157.56.162.53,0


Please wait. Getting VPS data...Expected 4 octets in '118.0' <class 'ipaddress.AddressValueError'>
Expected 4 octets in '119.0' <class 'ipaddress.AddressValueError'>
Expected 4 octets in '94.136.192' <class 'ipaddress.AddressValueError'>
Expected 4 octets in '0000' <class 'ipaddress.AddressValueError'>
Address cannot be empty <class 'ipaddress.AddressValueError'>
done


In [14]:
ip_result.browse_ti_results()

## Example - getting Whois information for Azure IPAddress requests.

In [19]:
(
    acc_result                      # the results object
    .azure_activity[["IPAddress"]]  # the property and the column we want
    .drop_duplicates()              # drop duplicates
    .mp_pivot.run(                  # run the pivot function IpAddress 'whois' function
        IpAddress.util.whois, column="IPAddress" 
    )
)

,nir,asn_registry,asn,asn_cidr,asn_country_code,asn_date,asn_description,query,nets,raw,referral,raw_referral
349,None,arin,8075,40.80.0.0/12,US,2015-02-23,"MICROSOFT-CORP-MSN-AS-BLOCK, US",40.91.208.65,"[{'cidr': '40.74.0.0/15, 40.80.0.0/12, 40.96.0.0/12, 40.125.0.0/17, 40.76.0.0/14, 40.124.0.0/16,...",None,None,None
350,None,arin,8075,104.208.0.0/13,US,2014-10-01,"MICROSOFT-CORP-MSN-AS-BLOCK, US",104.209.247.23,"[{'cidr': '104.208.0.0/13', 'name': 'MSFT', 'handle': 'NET-104-208-0-0-1', 'range': '104.208.0.0...",None,None,None
0,None,ripencc,12392,109.88.208.0/20,BE,2009-09-01,"ASBRUTELE VOO, BE",109.88.218.99,"[{'cidr': '109.88.208.0/20', 'name': 'TECTEO44', 'handle': 'JMA50-RIPE', 'range': '109.88.208.0 ...",None,None,None
246,None,arin,8075,157.56.0.0/16,US,1994-04-28,"MICROSOFT-CORP-MSN-AS-BLOCK, US",157.56.162.53,"[{'cidr': '157.56.0.0/14, 157.54.0.0/15, 157.60.0.0/16', 'name': 'MSFT-GFS', 'handle': 'NET-157-...",None,None,None
245,None,arin,8075,40.80.0.0/12,US,2015-02-23,"MICROSOFT-CORP-MSN-AS-BLOCK, US",40.83.137.154,"[{'cidr': '40.112.0.0/13, 40.125.0.0/17, 40.80.0.0/12, 40.76.0.0/14, 40.96.0.0/12, 40.120.0.0/14...",None,None,None
309,None,arin,8075,52.152.0.0/13,US,2015-11-24,"MICROSOFT-CORP-MSN-AS-BLOCK, US",52.154.174.165,"[{'cidr': '52.146.0.0/15, 52.145.0.0/16, 52.152.0.0/13, 52.148.0.0/14, 52.160.0.0/11', 'name': '...",None,None,None
336,None,arin,8075,52.152.0.0/13,US,2015-11-24,"MICROSOFT-CORP-MSN-AS-BLOCK, US",52.154.174.167,"[{'cidr': '52.145.0.0/16, 52.152.0.0/13, 52.160.0.0/11, 52.148.0.0/14, 52.146.0.0/15', 'name': '...",None,None,None
343,None,arin,NA,NA,US,2014-06-05,NA,2603:10b6:5:251::12,"[{'cidr': '2603:1000::/24', 'name': 'MSFT', 'handle': 'NET6-2603-1000-1', 'range': '2603:1000:: ...",None,None,None
341,None,arin,NA,NA,US,2014-06-05,NA,2603:10b6:5:3b9:cafe::47,"[{'cidr': '2603:1000::/24', 'name': 'MSFT', 'handle': 'NET6-2603-1000-1', 'range': '2603:1000:: ...",None,None,None
297,None,arin,NA,NA,US,2014-06-05,NA,2603:10b6:5:1f4:cafe::29,"[{'cidr': '2603:1000::/24', 'name': 'MSFT', 'handle': 'NET6-2603-1000-1', 'range': '2603:1000:: ...",None,None,None


# More information:

## Notebooklets and Pivots
[Notebooklets](https://msticnb.readthedocs.io/en/latest/)

[Pivot functions](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

## Notebook/MSTICPy configuration
[MSTICPy Configuration guide](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)

[ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)